In [1]:
import numpy as np
from mha021 import *

In [2]:
phi = 60 # [degrees]
L = 1.8  # [m]

c = np.cos(np.deg2rad(phi))
s = np.sin(np.deg2rad(phi)) # helper variables

# Element cooridnates
Ex = np.array([
    [0,       L*c],
    [L*c,   2*L*c],
    [2*L*c, 3*L*c],
    [3*L*c, 4*L*c],
    [L*c,   3*L*c],
    [0,    3*L*c],
    [4*L*c,   L*c]
])

Ey =np.array([  
              [0,    L*s],
       [L*s, 2*L*s],
     [2*L*s,   L*s],
       [L*s,   0],
       [L*s,   L*s],
       [0,     L*s],
       [0,     L*s]
])

fig = draw_discrete_elements(Ex, Ey)
fig.show()
# eldraw2(Ex, Ey) % Draw the truss structure

E = 210e9   # Young's modulus [Pa]
A = 2.0e-4   # cross-sectional area [m^2]
P = 13e3      # force magnitude [N]
sig_y = 250e6 # material yield limit [Pa] 

# Write you implementation below

In [4]:
print('a)')
Edofs = np.array([
    [1,2,5,6],
    [5,6,9,10],
    [9,10,7,8],
    [7,8,3,4],
    [5,6,7,8],
    [1,2,7,8],
    [3,4,5,6]
])

num_dofs = 10
num_el = 7

K = np.zeros((num_dofs, num_dofs))
f = np.zeros((num_dofs))

for el in range(num_el):
    dofs = Edofs[el, :]
    ex = Ex[el, :]
    ey = Ey[el, :]
    print(Ex[el, :])
    print(Ex[el, :])
    
    Ke = bar2e(ex, ey, E, A)
    assem(K, Ke, dofs)

f[9-1] = P
f[5-1] = -2 * P
f[6-1] = -P

bc_dofs = np.array([2, 3, 4])
bc_vals = np.zeros_like(bc_dofs)

a, r = solve_eq(K, f, bc_dofs, bc_vals)

Ed = extract_dofs(a, Edofs)
plot_deformed_bars(fig, Ex, Ey, Ed)
fig.show()

print(f'horizontal displacement at node 5 [mm]: {a[9-1] * 1e3}')

a)
[0.  0.9]
[0.  0.9]
[0.9 1.8]
[0.9 1.8]
[1.8 2.7]
[1.8 2.7]
[2.7 3.6]
[2.7 3.6]
[0.9 2.7]
[0.9 2.7]
[0.  2.7]
[0.  2.7]
[3.6 0.9]
[3.6 0.9]


horizontal displacement at node 5 [mm]: -2.3378563497936926


In [23]:
print('b)')

def FEM(P):
    K = np.zeros((num_dofs, num_dofs))
    f = np.zeros((num_dofs))

    for el in range(num_el):
        dofs = Edofs[el, :]
        ex = Ex[el, :]
        ey = Ey[el, :]
        
        Ke = bar2e(ex, ey, E, A)
        assem(K, Ke, dofs)

    f[9-1] = P
    f[5-1] = -2 * P
    f[6-1] = -P

    bc_dofs = np.array([2, 3, 4])
    bc_vals = np.zeros_like(bc_dofs)

    a, r = solve_eq(K, f, bc_dofs, bc_vals)
    Ed = extract_dofs(a, Edofs)
    
    N_list = np.zeros((num_el))
    for el in range(num_el):
        N_list[el] = bar2s(Ex[el, :], Ey[el, :], E, A, Ed[el, :])
        
    N_max = np.max(N_list)
    
    print(f'N_max: {N_max}')
    
    sigma = N_max / A

    return sigma

"""
i = 1
sigma = 0
while sigma < sig_y:
    new_P = P * (1 +  0.1 * i)
    print(f'\nP = {new_P}')
    sigma = FEM(new_P) 
    print(f'sigma: {sigma}')
    i += 1

print('iterative results:')
print(f'yielding at P = {new_P}')
"""

print('scaling results:')
print(f'P = {sig_y / FEM(P) * P}')

b)
scaling results:
N_max: 25766.660498395395
P = 25226.396724576647


In [ ]:
# problem 2

print('a)')